In [3]:
from general_json2yolo import convert_infolks_json as JY


In [4]:
name = "test"
files = r"C:\Users\excel\OneDrive\문서\dev\YearDeam_MAKE\1.data\4.dataSet\json\0.json"
img_path = r"C:\Users\excel\OneDrive\문서\dev\YearDeam_MAKE\1.data\4.dataSet\img"
JY(name, files, img_path)

TypeError: list indices must be integers or slices, not str